In [2]:
import asyncio

async def handle_client(reader, writer):
    request = None
    while request != 'quit':
        request = (await reader.read(255)).decode('utf8')
        response = str(eval(request)) + '\n'
        writer.write(response.encode('utf8'))
        await writer.drain()
    writer.close()

async def run_server():
    server = await asyncio.start_server(handle_client, 'localhost', 15555)
    async with server:
        await server.serve_forever()

In [ ]:
await run_server()

In [ ]:
import asyncio

async def handle_echo(reader, writer):
    data = await reader.read(100)
    message = data.decode()
    addr = writer.get_extra_info('peername')

    print(f"Received {message!r} from {addr!r}")

    print(f"Send: {message!r}")
    writer.write(data)
    await writer.drain()

    print("Close the connection")
    writer.close()
    await writer.wait_closed()

async def main():
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 1234)

    addrs = ', '.join(str(sock.getsockname()) for sock in server.sockets)
    print(f'Serving on {addrs}')

    async with server:
        await server.serve_forever()

await main()

Serving on ('127.0.0.1', 1234)


In [2]:
import asyncio
import socket

HOST, PORT = 'localhost', 1234

class UdpReceiver(asyncio.DatagramProtocol):
    def __init__(self):
        super().__init__()
    
    def connection_made(self, transport: asyncio.DatagramTransport) -> None:
        self.transport = transport
    
    def datagram_received(self, data: bytes, addr) -> None:
        print(f"Received Message: {data}")



In [3]:
loop = asyncio.get_event_loop()
t = loop.create_datagram_endpoint(UdpReceiver, local_addr=('0.0.0.0', PORT))
loop.run_until_complete(t)
loop.run_forever()

RuntimeError: This event loop is already running